In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [2]:
data = pd.read_csv('Train.csv')

In [3]:
data.head()

,session_id,session_number,client_agent,device_details,date,purchased,added_in_cart,checked_out,time_spent
0,57f879e70d3c5fc2a98102d64c9fd84e,715,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,Desktop - Chrome,1/22/2020,1,0,0,236.886
1,a5442b0c7c33d0a811e7661e556b2de8,55,Product/8.0 iPhone/8.1.3,iPhone - iOS,2/27/2020,1,0,0,1277.455
2,305cb1486ed8610c00b37007926cb2c4,11,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like ...,iPhone - MobileWeb,8/1/2019,0,0,0,21.704
3,f2c1ecc9993f0071df91ba178450498c,2794,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Desktop - IE,12/30/2019,0,1,0,455.201
4,e460830ae295e55d2216ebdc761ab9a6,3674,Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like ...,iPhone - Web,9/10/2019,0,0,0,1164.877


In [ ]:
data['os'] = data['client_agent'].str.split("/",n = 1, expand = True)[1].str.split(' ',n = 1, expand = True)[1]
data.os.value_counts()

In [ ]:
data['client'] = data['client_agent'].str.split("/",n = 1, expand = True)[0]
data['c_version'] = data['client_agent'].str.split("/",n = 1, expand = True)[1].str.split(' ',n = 1, expand = True)[0]
data.c_version.value_counts()
data.isna().sum()

In [13]:
# Create a label encoder object 
def MultiLabelEncoder(columnlist,dataframe):
    for i in columnlist:
        labelencoder_X=preprocessing.LabelEncoder()
        dataframe[i]=labelencoder_X.fit_transform(dataframe[i])
    return dataframe


def preprocess(data):
    data['client'] = data['client_agent'].str.split("/",n = 1, expand = True)[0]
    data["client"] = data["client"].astype("str")
    data['client'].fillna(data['client'].mode(), inplace=True)
    data['c_version'] = data['client_agent'].str.split("/",n = 1, expand = True)[1].str.split(' ',n = 1, expand = True)[0]
    data["c_version"] = data["c_version"].astype("str")
    data['c_version'].fillna(data['c_version'].mode(), inplace=True)
    data['device'] = data['device_details'].str.split("-",n = 1, expand = True)[0]
    data['browser'] = data['device_details'].str.split("-",n = 1, expand = True)[1]
    data.date = pd.to_datetime(data.date)
    data['year'] = data.date.dt.year
    data['month'] = data.date.dt.month
    data['day'] = data.date.dt.day
    data['dayofweek'] = data.date.dt.dayofweek
    data['month_start'] = data.date.dt.is_month_start
    data['month_end'] = data.date.dt.is_month_end
    cols = ['client','c_version','purchased','added_in_cart','checked_out','device','browser','month','day','dayofweek','month_start','month_end','time_spent']
    inn = data[cols]
    columnlist = ['client','device','browser','c_version','month_start','month_end']
    inn = MultiLabelEncoder(columnlist,inn)
    return inn

pre_data = preprocess(data)


/Users/subbuvidyasekar/Soft/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
pre_data.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0.1, 0.9))
scaled = scaler.fit_transform(pre_data)
scal = pd.DataFrame(data = scaled, columns = pre_data.columns)

In [14]:
def model():
    y = pre_data[['time_spent']]
    x = pre_data.drop(['time_spent'], axis = 1)
    x_value = np.array(x)
    y_value = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(x_value, y_value,test_size=0.1,random_state=42)
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    predd = lin_reg.predict(X_test)
    #print(mean_squared_log_error(predd, y_test)) 
    return lin_reg
lin_reg = model()

In [15]:
def preprocess_test(data):
    data['client'] = data['client_agent'].str.split("/",n = 1, expand = True)[0]
    data["client"] = data["client"].astype("str")
    data['client'].fillna(data['client'].mode(), inplace=True)
    data['c_version'] = data['client_agent'].str.split("/",n = 1, expand = True)[1].str.split(' ',n = 1, expand = True)[0]
    data["c_version"] = data["c_version"].astype("str")
    data['c_version'].fillna(data['c_version'].mode(), inplace=True)
    data['device'] = data['device_details'].str.split("-",n = 1, expand = True)[0]
    data['browser'] = data['device_details'].str.split("-",n = 1, expand = True)[1]
    data.date = pd.to_datetime(data.date)
    data['year'] = data.date.dt.year
    data['month'] = data.date.dt.month
    data['day'] = data.date.dt.day
    data['dayofweek'] = data.date.dt.dayofweek
    data['month_start'] = data.date.dt.is_month_start
    data['month_end'] = data.date.dt.is_month_end
    cols = ['client','c_version','purchased','added_in_cart','checked_out','device','browser','month','day','dayofweek','month_start','month_end']
    inn = data[cols]
    columnlist = ['device','browser','client','c_version','month_start','month_end']
    inn = MultiLabelEncoder(columnlist,inn)
    return inn

In [16]:
test = pd.read_csv('Test.csv')
sub = pd.read_csv("Sample Submission.csv")
pre_test = preprocess_test(test)
#scaler = MinMaxScaler(feature_range=(0.1, 0.9))
#scaled_test = scaler.fit_transform(pre_test)

/Users/subbuvidyasekar/Soft/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [17]:
pred = lin_reg.predict(pre_test)
pred

array([[1217.98725484],
       [ 486.58735136],
       [1481.71969948],
       ...,
       [ 456.40526061],
       [1231.19673385],
       [1156.51534389]])

In [18]:
sub['time_spent'] = pred
(sub < 0).values.any()

True

In [19]:
sub['time_spent'] = sub['time_spent'].abs()
sub.to_csv("Submission.csv", index = False)

In [ ]:
type(pred)